In [27]:
import sys
sys.path.append("../")

import csv
import pandas as pd
import numpy as np
import json
from itertools import repeat, chain
import utils

## Running to Collect ACS Data using Census API

In [48]:
# Lets set up our requests session. You can use teh creaet session utility or just pass the requests object
session = utils.create_session()

## State codes for all states & Washington DC
## As listed here - https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
#states = [i for i in range(1,57) if i not in [3, 7, 14, 43, 52]]
states = [ 4, 44,  10, 34, 44, 34]

In [49]:
## These codes and endpoints are from viewing the Census API documentation once you identify the correct table
## Here's a google doc of the codes we want to use: 

codes = {'acs/acs5':[  "B15003_001E", "B15003_015E","B15003_016E","B15003_017E","B15003_018E",
                    "B15003_019E", "B15003_020E", "B15003_021E", "B15003_022E", "B15003_023E", "B15003_024E", 
                    "B15003_025E",  "B05004_001E", "B05004_007E", "B09010_001E", "B09010_002E",
                    "B19001_001E",'B19001_002E', 'B19001_003E', 'B19001_004E', 'B19001_005E', 'B19001_006E', 
                    'B19001_007E', 'B19001_008E', 'B19001_009E', 'B19001_010E', 'B19001_011E', 'B19001_012E', 
         'B19001_013E','B19001_014E', 'B19001_015E', 'B19001_016E', 'B19001_017E'],
         
         # "B01003_001E", "B01002_001E", "B01002_002E", "B01002_003E",
         #"B28002_001E", "B28002_002E", "B28002_003E", "B28002_004E", "B28002_005E", 
#                     "B28002_006E", "B28002_007E", "B28002_008E", "B28002_009E", "B28002_010E", "B28002_011E", 
#                     "B28002_012E", "B28002_013E", "B28003_001E", "B28003_002E", "B28003_003E", "B28003_004E",
#                     "B28003_005E", "B28003_006E",
         #"S2301_C03_001E", "S1701_C02_001E", "S1701_C01_001E", "S1901_C01_012E",
                            # "S1101_C01_001E", "S1101_C01_002E", "S1101_C01_004E", "S2701_C03_011E","S2701_C03_012E", "S2701_C03_013E",
        'acs/acs5/subject' : [
                              'S0101_C01_001E', 'S0101_C01_002E', 'S0101_C01_003E',
                              'S0101_C01_004E', 'S0101_C01_005E', 'S0101_C01_006E', 'S0101_C01_007E', 'S0101_C01_008E', 
                              'S0101_C01_009E', 'S0101_C01_010E', 'S0101_C01_011E', 'S0101_C01_012E', 'S0101_C01_013E', 
                              'S0101_C01_014E', 'S0101_C01_015E', 'S0101_C01_016E', 'S0101_C01_017E', 'S0101_C01_018E', 
                              'S0101_C01_019E', 'S1810_C01_001E', 'S1810_C01_019E', 'S1810_C01_029E', 'S1810_C01_039E',
                             'S1810_C01_047E', 'S1810_C01_055E', 'S1810_C01_063E', 'S1810_C02_001E','S1101_C01_010E',
                             'S1101_C01_009E']}
         #'DP04_0142PE', 'DP04_0141PE', "DP04_0089E",
#         'acs/acs5/profile' : [ "DP05_0077PE", "DP05_0071PE",
#                              "DP05_0078PE", "DP05_0079PE", "DP05_0080PE", "DP05_0081PE", "DP05_0082PE", "DP05_0083PE"
#                              ]}

acs_endpoints = ['acs/acs5', 'acs/acs5/subject'] 


dfs = []
outputs6 = utils.gather_results(session, acs_endpoints, states, codes, dfs)

https://api.census.gov/data/2019/acs/acs5?get=NAME,B15003_001E,B15003_015E,B15003_016E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B05004_001E,B05004_007E,B09010_001E,B09010_002E,B19001_001E,B19001_002E,B19001_003E,B19001_004E,B19001_005E,B19001_006E,B19001_007E,B19001_008E,B19001_009E,B19001_010E,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E&for=tract:*&in=state:04&key=613f835e522e3664c4ef55859c35824cf26dadc3
https://api.census.gov/data/2019/acs/acs5/subject?get=NAME,S0101_C01_001E,S0101_C01_002E,S0101_C01_003E,S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,S0101_C01_018E,S0101_C01_019E,S1810_C01_001E,S1810_C01_019E,S1810_C01_029E,S1810_C01_039E,S1810_C01_047E,S1810_C01_055E,S1810_C01_063E,S1810_C02_001E,S1101_C01_010E,S1101

In [50]:
outputs6.shape

(6252, 66)

In [51]:
np.unique(outputs6.state).shape

(4,)

In [52]:
outputs6.state.value_counts()

34    4020
04    1526
44     488
10     218
Name: state, dtype: int64

In [53]:
outputs6 = outputs6.replace(-1, np.nan)
outputs6 = outputs6.replace(-666666666, np.nan)
outputs6 = outputs6.replace(-888888888, np.nan)
outputs6 = outputs6.replace(-666666666.0, np.nan)

In [54]:

pd.set_option('display.max_rows', None)
outputs6.isnull().sum()

NAME                 0
B15003_001E        244
B15003_015E        244
B15003_016E        244
B15003_017E        244
B15003_018E        244
B15003_019E        244
B15003_020E        244
B15003_021E        244
B15003_022E        244
B15003_023E        244
B15003_024E        244
B15003_025E        244
B05004_001E        244
B05004_007E        244
B09010_001E        244
B09010_002E        244
B19001_001E        244
B19001_002E        244
B19001_003E        244
B19001_004E        244
B19001_005E        244
B19001_006E        244
B19001_007E        244
B19001_008E        244
B19001_009E        244
B19001_010E        244
B19001_011E        244
B19001_012E        244
B19001_013E        244
B19001_014E        244
B19001_015E        244
B19001_016E        244
B19001_017E        244
state                0
county               0
tract                0
S0101_C01_001E    2010
S0101_C01_002E    2010
S0101_C01_003E    2010
S0101_C01_004E    2010
S0101_C01_005E    2010
S0101_C01_006E    2010
S0101_C01_0

In [55]:
outputs6.head()

,NAME,B15003_001E,B15003_015E,B15003_016E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,...,S1810_C01_001E,S1810_C01_019E,S1810_C01_029E,S1810_C01_039E,S1810_C01_047E,S1810_C01_055E,S1810_C01_063E,S1810_C02_001E,S1101_C01_010E,S1101_C01_009E
0,"Census Tract 22, Coconino County, Arizona",5191,4,120,501,191,167,659,605,1385,...,6868,-888888888,-888888888,-888888888,-888888888,-888888888,-888888888,892,30.7,2684
1,"Census Tract 23, Coconino County, Arizona",5016,51,44,1157,166,555,1382,280,740,...,6182,-888888888,-888888888,-888888888,-888888888,-888888888,-888888888,840,12.9,2582
2,"Census Tract 9422.01, Coconino County, Arizona",2150,107,42,771,52,81,320,160,107,...,3712,-888888888,-888888888,-888888888,-888888888,-888888888,-888888888,790,38.3,1006
3,"Census Tract 9422.02, Coconino County, Arizona",1902,169,36,705,34,80,279,83,63,...,3594,-888888888,-888888888,-888888888,-888888888,-888888888,-888888888,774,46.8,873
4,"Census Tract 9449, Coconino County, Arizona",3056,95,17,998,82,155,536,385,204,...,4904,-888888888,-888888888,-888888888,-888888888,-888888888,-888888888,486,47.8,1239


In [56]:
outputs6[outputs6.S1810_C01_029E.isna()]["state"].value_counts()

34    2010
Name: state, dtype: int64

In [57]:
outputs6[outputs6.B19001_017E.isna()]["state"].value_counts()

44    244
Name: state, dtype: int64

In [58]:
x = outputs6[~outputs6.B19001_017E.isna()]
x[x.B19001_017E.isna()]["state"].value_counts()

Series([], Name: state, dtype: int64)

In [59]:
x = x[~x.S1810_C01_029E.isna()]
x[x.S1810_C01_029E.isna()]["state"].value_counts()

Series([], Name: state, dtype: int64)

In [60]:
x.to_csv("../data/additional_data_v4.csv", index=False)